### Introduction

This project is aimed at extracting the data of hadoop contributors from github api and storing it into mysql database.  

### Variables extracted
- Name
- Id
- Location
- Hiring status
- Bio
- Following on github
- Contact information
- Profile URL

### Business Outcome:
Data scraped from the internet can be a valuable resource for statistical analysis and research purposes. By storing the data of all contributors on github, a company can easily access the details of potential employees. This information can be used by recruitment team to filter candidates who are hireable and has been contributing to the hadoop space. 
Instead of going through the profile of each and every individual, the sql database can be queried to access the needed information. 

### Tools and Technologies Used
- MySQL
- Beautiful soup



Loading libraries

In [ ]:
import pymysql
import warnings
import requests
import json
import codecs
from bs4 import BeautifulSoup
import time
from datetime import datetime


Urban dictionary is not a website but it is providing a json object in return


In [ ]:
url = "https://api.github.com/repos/apache/hadoop/contributors";
header={''}

page = requests.get(url,headers=header,params={'per_page': 100},auth=('username','token'))
doc = BeautifulSoup(page.content, 'html.parser')
json_dict = json.loads(str(doc))


Fetching profile URLs of contributor to store in a list

In [ ]:
profile_url=[]

try:   
    for i in range(0,100):
        profile_url.append((json_dict[i]['url']))
except:
    print(i)



Fecthing the cotributor details from the profile URLs saved in the list

In [6]:
user_id_=[]
login_=[]
location_=[]
email_=[]
hireable_=[]
bio_=[]
twitter_username_=[]
public_repos_=[]
public_gists_=[]
followers_=[]
following_=[]
created_at_=[]

for i in range(0,100):
        page = requests.get(profile_url[i],headers=header,auth=('username','auth_token'))
        doc = BeautifulSoup(page.content, 'html.parser')
        json_dict = json.loads(str(doc))
        
        user_id_.append(json_dict["id"])
        login_.append(json_dict['login'])
        location_.append(json_dict["location"])
        email_.append(json_dict["email"])
        hireable_.append(json_dict["hireable"]) 
        bio_.append(json_dict["bio"])
        twitter_username_.append(json_dict["twitter_username"])
        public_repos_.append(json_dict["public_repos"])
        public_gists_.append(json_dict["public_gists"])
        followers_.append(json_dict["followers"])
        following_.append(json_dict["following"])
        if json_dict["created_at"] is not None:
            json_dict["created_at"] = datetime.strptime(json_dict["created_at"],"%Y-%m-%dT%H:%M:%SZ")
            created_at_.append(json_dict["created_at"])
        else: 
            created_at_.append(json_dict["created_at"])
        


Function to create a mysql database and a table

In [14]:
def create_sql_table(SQL_TABLE_URBAN, SQL_TABLE_URBAN_DEF):
    try:

        #connect to server
        conn = pymysql.connect(host='localhost', user = '', password = ' ')
        cursor = conn.cursor()

        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);

        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + SQL_TABLE_URBAN + " " + SQL_TABLE_URBAN_DEF + ";";
        print(query)
        cursor.execute(query);
        cursor.close()
        conn.close()
        return

    except IOError as e:
            print(e)


Specify the name and structure of the table

In [15]:
SQL_DB = "DDR_Github"
TERM = "blog"

SQL_TABLE_URBAN = "userprofile"
SQL_TABLE_URBAN_DEF = "(" + \
        "ind INT NOT NULL AUTO_INCREMENT PRIMARY KEY" + \
        ",userid INT " + \
        ",login VARCHAR(50)" + \
        ",location VARCHAR(100)" + \
        ",email VARCHAR(30)" + \
        ",hireable BOOLEAN" + \
        ",bio VARCHAR(1000)" + \
        ",twitter_username VARCHAR(30)" + \
        ",public_repos INT" + \
        ",public_gists INT" + \
        ",followers INT" + \
        ",following INT" + \
        ",created_at DATETIME" + \
        ")"

create_sql_table(SQL_TABLE_URBAN,SQL_TABLE_URBAN_DEF)

CREATE DATABASE IF NOT EXISTS DDR_Github
CREATE TABLE IF NOT EXISTS DDR_Github.userprofile (ind INT NOT NULL AUTO_INCREMENT PRIMARY KEY,userid INT ,login VARCHAR(50),location VARCHAR(100),email VARCHAR(30),hireable BOOLEAN,bio VARCHAR(1000),twitter_username VARCHAR(30),public_repos INT,public_gists INT,followers INT,following INT,created_at DATETIME);


Inserting rows in the table


In [16]:
try:
    #connect to server
    conn = pymysql.connect(host='localhost', user = '', password = '', database = '')
    cursor = conn.cursor()   
        
        
    #standard version
#     prepared_stmt = "INSERT INTO " + SQL_TABLE_URBAN + "(userid,login,location,email,hireable,bio, twitter_username,public_repos,public_gists,followers,following ,created_at ) values ('{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}');" 
    
    #parametrized version
    parameterized_stmt = "INSERT INTO " + SQL_TABLE_URBAN + "(userid,login,location,email,hireable,bio, twitter_username,public_repos,public_gists,followers,following ,created_at ) values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) ;"
    
    for i in range(0,100):        
        user_id=user_id_[i]
        login=login_[i]
        location=location_[i]
        email=email_[i]
        hireable=hireable_[i]
        bio=bio_[i]
        twitter_username=twitter_username_[i]
        public_repos=public_repos_[i]
        public_gists=public_gists_[i]
        followers=followers_[i]
        following=following_[i]
        created_at=created_at_[i]
        
#         stmt = prepared_stmt.format(user_id, login, location, email, ((hireable) if hireable else 0), codecs.escape_encode(str.encode("'{}'".format(bio) if bio else "NULL"))[0].decode(), twitter_username, public_repos, 
#                                     public_gists, followers, following, created_at )
        

        
#         cursor.execute(stmt)
        # parameterized version
        cursor.execute(parameterized_stmt, (user_id, login, location, email, ((hireable) if hireable else 0), codecs.escape_encode(str.encode("'{}'".format(bio) if bio else "NULL"))[0].decode(), twitter_username, public_repos, public_gists, followers, following, created_at))
        
        
    conn.commit()
    cursor.close()
    conn.close()
except IOError as e:
    print(e)




For userid I chose integer as this is an integer. Login is a unique username that gets created with github. This can be a combination of string and integers so varchar is the right choice for it. 
For location and email, it is a mix of strings and numerals. Special characters such as #, @ can also be present so choosing varchar is the right choice. Hireable is a boolean True or False so boolean data type is used for it. 
Twitter username is again unique as github login which can be a mix of strings and numbers so varchar datatype is chosen for it. 
Public repos, public gists,followers and following are numbers hence integer data type is used. 
For created at, it is a datetime so datetime datatype is used to store the time and date of creation.


Indexing the table for easier and faster access

In [ ]:
SQL_DB = "DDR_Github"
TERM = "blog"
SQL_TABLE_URBAN = "userprofile"

try:
    #connect to server
    conn = pymysql.connect(host='localhost', user = '', password = '', database = '')
    cursor = conn.cursor()    
        
    #standard version
    prepared_stmt1 = "CREATE INDEX indexing_login ON "+ SQL_DB +"."+ SQL_TABLE_URBAN +"(login);"
    prepared_stmt2 = "CREATE INDEX ind_loc ON "+ SQL_DB +"."+ SQL_TABLE_URBAN +"(location);"
    prepared_stmt3 = "CREATE INDEX ind_hire ON "+ SQL_DB +"."+ SQL_TABLE_URBAN +"(hireable);"
      
    cursor.execute(prepared_stmt1)
    cursor.execute(prepared_stmt2)
    cursor.execute(prepared_stmt3)     
    conn.commit()
    cursor.close()
    conn.close()
except IOError as e:
    print(e)



I went with indexing as index creastes a way for faster lookup. The search doesn't have to be done on each and every row in the data if there is index. Algorithms such as binary search, merge sort takes into account the index making it easier to divide and conquer. SQL uses binary search which makes the effort to look for elements significantly small. Hashcodes are used for indexing string objects. 